### This noteboook generates abstractive summarization on research articles related to coronaviruses. [CORD-19: The Covid-19 Open Research Dataset](https://allenai.org/data/cord-19).

### The code uses BERT-Base uncased and [BertAbs](https://github.com/nlpyang/PreSumm) model.

In [1]:
import glob
import json
import pandas as pd
import os
import spacy
import re
import string
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from langdetect import detect
import itertools
from numba import jit, cuda
from summa import summarizer

## Data loading and cleaning
   1. Load papers into a dataframe.
   2. Filter papers without an abstract or body text.
   3. Filter non-english papers.

In [2]:
%%time

path = '/home/ubuntu/data/*.json'
files = glob.glob(path)
papers = []
for file in files:
    with open(file) as json_file:
            text = json.load(json_file)
            papers.append([text['paper_id'], text['bodytext'], text['abstract']])
data = pd.DataFrame(papers, columns = ['paper_id', 'bodytext', 'abstract'])

#get the lengths of texts
data['len_bt'] = data.bodytext.map(lambda x: len(x.split(" ")))
data['len_ab'] = data.abstract.map(lambda x: len(x.split(" ")))

#filter papers with certain word length
data.query('len_bt <= 10000 and len_bt>=1000 and len_ab <= 500 and len_ab >=50', inplace = True)

#detect languages of texts to filter out non-english papers
data['bt_lang'] = data.bodytext.map(lambda x: detect(x))
data['ab_lang'] = data.abstract.map(lambda x: detect(x))

#use only english papers
data = data[(data['bt_lang'] == 'en') & (data['ab_lang'] == 'en')]

CPU times: user 6min 9s, sys: 1.74 s, total: 6min 11s
Wall time: 6min 11s


#### Split the data into train and test datasets

In [3]:
train, test = train_test_split(data, test_size = 0.2, random_state = 42)
train = train.reset_index()
test = test.reset_index()
len(train), len(test)

(6130, 1533)

#### Since BERT base model only takes 512 tokens and the text length of our papers is greater than that, extractive summaries limited to word length 500 are obtained using summa package. 
#### These summaries are written in files and are given as input to the BertAbs model. 

In [4]:
import os
def create_story_files(df,num_file,corpus_type,filename=None):
    for i in range(num_file):
        doc = df.iloc[i]['bodytext'] + '\n'*2 + '@abstract' + '\n'*2 + df.iloc[i]['abstract']
        file_name = os.path.join(filename,('paper_' + corpus_type + '_' + str(i) + '.story'))
        #print(file_name)
        with open(file_name,'w+') as story_file:
            story_file.write(doc)

def create_ext_summary_files(df,num_file,corpus_type,filename=None):
    for i in range(num_file):
        ext_text = summarizer.summarize(df.iloc[i]['bodytext'], words = 500)
        ext_text = ext_text.replace('\n', '')
        doc = ext_text + '\n'*2 + '@abstract' + '\n'*2 + df.iloc[i]['abstract']
        #print(doc)
        file_name = os.path.join(filename,('paper_' + corpus_type + '_' + str(i) + '.story'))
        with open(file_name,'w+') as story_file:
            story_file.write(doc)


In [5]:
n_train = len(train)
n_test = len(test)
# create_story_files(train,n_train,corpus_type='train',filename='/home/ubuntu/BERT/covid_data')
# create_story_files(test,n_test,corpus_type = 'test',filename='/home/ubuntu/BERT/covid_data')

create_ext_summary_files(train,n_train,corpus_type='train',filename='/home/ubuntu/BERT/covid_extracted_summaries')
create_ext_summary_files(test,n_test,corpus_type = 'test',filename='/home/ubuntu/BERT/covid_extracted_summaries')

#### The extracted summaries are pre-processed by splitting sentences and tokenization and stored in seperate json files.

In [ ]:
os.chdir("/home/ubuntu/stanford-corenlp-full-2017-06-09")
os.getcwd()
!python /home/ubuntu/PreSumm/src/preprocess.py -mode tokenize -raw_path /home/ubuntu/BERT/covid_extracted_summaries -save_path /home/ubuntu/BERT/merged_stories_tokenized  -log_file /home/ubuntu/BERT/logs/covid.log
os.chdir("/home/ubuntu/")
os.getcwd()

#### The tokens in json files are divided into source (body text) and target tokens (abstract) and combined into a single json file each for train and test dataset. 

In [ ]:
!python /home/ubuntu/PreSumm/src/preprocess.py -mode custom_format_to_lines -raw_path /home/ubuntu/BERT/merged_stories_tokenized -save_path /home/ubuntu/BERT/json_data/covid_data -n_cpus 1 -use_bert_basic_tokenizer false -log_file /home/ubuntu/BERT/logs/covid.log

#### Further pre-processing steps are applied to the text and saved in a PyTorch file
 - Three special tokens, [CLS] at the beginning of a text and [SEP] at the end of every sentence and [PAD] for padding are added.
 - Token, segmentation and position embeddings are added. 

In [8]:
!python /home/ubuntu/PreSumm/src/preprocess.py -mode custom_format_to_bert -raw_path /home/ubuntu/BERT/json_data/ -save_path /home/ubuntu/BERT/covid_bert_data  -lower -n_cpus 1 -log_file /home/ubuntu/BERT/logs/cnndm.log 

.train.0.json
/home/ubuntu/BERT/json_data/covid_data.train.0.json
covid_data.train.0.json
[('train', '/home/ubuntu/BERT/json_data/covid_data.train.0.json', Namespace(dataset='', log_file='/home/ubuntu/BERT/logs/cnndm.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='custom_format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='/home/ubuntu/BERT/json_data/', save_path='/home/ubuntu/BERT/covid_bert_data', select_mode='greedy', shard_size=9000, use_bert_basic_tokenizer=False), '/home/ubuntu/BERT/covid_bert_data/covid_data.train.0.bert.pt')]
[2021-01-22 14:42:33,249 INFO] loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c12

#### BertAbs model is trained
 - Encoder is pretrained [BERTSUM](https://arxiv.org/pdf/1903.10318.pdf) model
 - Decoder is a 6 layered Transformer initialized randomly
 - Two Adam optimizers (0.9, 0.999) are used for the encoder and decoder seperately. This is because the encoder is pretrained while decoder is not. This can make fine-tuning unstable.
 - learning rate for encoder is 0.002 and decoder is 0.1 because the encoder (which is pretrained) needs to fine-tuned at a smaller rate. 
 - model is trained for total 100,000 steps and checkppoints are saved every 10,000 steps

In [11]:
!python ~/PreSumm/src/train.py  -task abs -mode train -bert_data_path ~/BERT/covid_bert_data/covid_data -dec_dropout 0.2  -model_path ~/BERT -sep_optim true -lr_bert 0.002 -lr_dec 0.2 -save_checkpoint_steps 10000 -batch_size 140 -train_steps 200000 -report_every 50 -accum_count 5 -use_bert_emb true -use_interval true -warmup_steps_bert 20000 -warmup_steps_dec 10000 -max_pos 500 -visible_gpus 0  -log_file /home/ubuntu/BERT/logs/abs_bert_covid

[2021-01-22 14:47:31,783 INFO] Namespace(accum_count=5, alpha=0.6, batch_size=140, beam_size=5, bert_data_path='/home/ubuntu/BERT/covid_bert_data/covid_data', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='/home/ubuntu/BERT/logs/abs_bert_covid', lr=1, lr_bert=0.002, lr_dec=0.2, max_grad_norm=0, max_length=150, max_pos=500, max_tgt_len=140, min_length=15, mode='train', model_path='/home/ubuntu/BERT', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=50, report_rouge=True, result_path='../results/cnndm', save_checkpoint_steps=10000, seed=666, sep_optim=True, share_emb=False, task='abs

#### Model is tested on the test dataset and rouge scores are calculated
#### The rouge score at 100,000 step
1. ROUGE-F(1/2/3): 31.75/11.11/19.61
2. ROUGE-R(1/2/3): 23.39/8.28/14.54


In [32]:
!python ~/PreSumm/src/train.py -task abs -mode test -model_path ~/BERT -test_from ~/BERT/model_step_100000.pt -batch_size 100 -test_batch_size 100 -bert_data_path ~/BERT/covid_bert_data/covid_data -temp_dir ~/BERT/temp -log_file ~/BERT/logs/abs_bert_covid  -sep_optim true -use_interval true -visible_gpus 0 -max_pos 512 -max_length 200 -alpha 0.82 -min_length 10 -result_path ~/BERT/logs/abs_bert_covid

[2021-01-25 00:22:18,764 INFO] Loading checkpoint from /home/ubuntu/BERT/model_step_100000.pt
Namespace(accum_count=1, alpha=0.82, batch_size=100, beam_size=5, bert_data_path='/home/ubuntu/BERT/covid_bert_data/covid_data', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='/home/ubuntu/BERT/logs/abs_bert_covid', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=10, mode='test', model_path='/home/ubuntu/BERT', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='/home/ubuntu/BERT/logs/abs_bert_covi

#### Print predicted summaries

#### Observations
1. Some predicted summaries are exactly the same as the abstract. This maybe due to the presence of some papers that have the same abstact and bodytext in the training set. 
2. The predicted summaries contain some of the important keywords from the abstracts. However, the prediction still could be improved further more. 

In [17]:
with open('/home/ubuntu/BERT/logs/abs_bert_covid.100000.gold','r') as s:
    summary = s.readlines()

for i in range(2):
    print(summary[i])

respiratory viral infections can trigger asthma attack which may lead to sever morbidity . in this report , using molecular methods , we show the chronological association between human coronavirus - hku1 infection and asthma exacerbation in a two years and seven months old asthmatic girl who was not under treatment and was otherwise healthy .

the ebola virus disease outbreak in west africa has prompted significant progress in responding to the clinical needs of patients affected by emerging infectious disease outbreaks . among the noteworthy successes of vaccine trials , and the commendable efforts to implement clinical treatment trials during ebola outbreaks , we should also focus on strengthening the collection and curation of epidemiological and observational data that can improve the conception and design of clinical research .



#### Print abstracts from papers

In [18]:
with open('/home/ubuntu/BERT/logs/abs_bert_covid.100000.candidate','r') as s:
    cand = s.readlines()

for i in range(2):
    print(cand[i])

respiratory viral infections can trigger asthma attack which may lead to sever morbidity . in this report , using molecular methods , we show the chronological association between human coronavirus - hku1 infection and asthma exacerbation in a two years and seven months old asthmatic girl who was not under treatment and was otherwise healthy .

background : continuous outbreaks of infectious diseases of ebola virus disease ( evd ) underscores the need for improved diagnostics and vaccines . while many of these pathogens are zoonotic , there is a paucity of data on the epidemiology of evd cases in the republic of congo , the world health organization ( who ) . evd outbreaks in these countries have been infected with evd and several of them . this review summarizes the current status of these novel therapeutics , and evaluate the role of non-infectious diseases in these outbreaks .

